In [1]:
import os
#os.environ['CUDA_VISIBLE_DEVICES'] = '1'
import sys
sys.path.insert(0, "../")
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision.models import MobileNetV2
from importlib import reload
import dataset
reload(dataset)
from models import ModelWrapper_TorchVisionNet
from dataset import VoiceAntiSpoofDataset
from utils import read_fromBaseline, read_scipy
from Metrics import compute_err

#import DftSpectrogram
from librosa.feature import mfcc
import glob


from kekas import Keker, DataOwner
from kekas.callbacks import Callback, ProgressBarCallback
from kekas.metrics import accuracy



/home/vfilin/anaconda3/envs/idrnd/lib/python3.6/site-packages/kekas/keker.py:9: UserWarning: Error 'No module named 'apex''' during importing apex library. To use mixed precison you should install it from https://github.com/NVIDIA/apex
  warnings.warn(f"Error '{e}'' during importing apex library. To use mixed precison"


In [2]:
MN2 = MobileNetV2()
MN2.classifier[1] = nn.Linear(1280, 2, bias=False)
model = ModelWrapper_TorchVisionNet(MN2).to('cuda')
sum(p.numel() for p in model.parameters() if p.requires_grad)

2226438

In [3]:
dataset_dir = '/home/vfilin/Desktop/voice_as_contest/Training_Data/'
print("Num samples:", len(glob.glob(os.path.join(dataset_dir, '**/*.wav'), recursive=True)))
dataset = VoiceAntiSpoofDataset(dataset_dir, 'all', read_scipy, transform=[lambda x: mfcc(x),  
                                lambda x: x[None, ...].astype(np.float32)])
dataset_val_dir = '/home/vfilin/Desktop/voice_as_contest/validationASV/'
dataset_val = VoiceAntiSpoofDataset(dataset_val_dir, 'all', read_scipy, transform=[lambda x: mfcc(x),  
                                lambda x: x[None, ...].astype(np.float32)])
batch_size = 24
num_workers = 12
#dataset.data = dataset.data[:48] 
#dataset.labels = dataset.labels[:48]
#dataset_val.data = dataset_val.data[:48] + dataset_val.data[-24:]
#dataset_val.labels = dataset_val.labels[:48] + dataset_val.labels[-24:]
dataloader = DataLoader(dataset, batch_size=batch_size, num_workers=num_workers, shuffle=True)
val_dl = DataLoader(dataset_val, batch_size=batch_size, num_workers=num_workers, shuffle=False)
len(dataset), len(dataset_val), len(np.unique(dataset.data))

Num samples: 50000
10322 qq
39678 qqq
1507 qq
1507 qqq


(50000, 3014, 50000)

In [4]:
dataowner = DataOwner(dataloader, val_dl, None)
weights = torch.tensor([5, 1.25])
weights = weights / weights.sum()
weights = weights.to('cuda')
criterion = nn.CrossEntropyLoss(weight=weights)

In [12]:
def step_fn(model: torch.nn.Module,
            batch: torch.Tensor) -> torch.Tensor:
    """Determine what your model will do with your data.

    Args:
        model: the pytorch module to pass input in
        batch: the batch of data from the DataLoader

    Returns:
        The models forward pass results
    """
    
    voice = batch['data']
    return model(voice)
class ScoreCallback(Callback):
    def __init__(self, preds_key, target_key, metric_fn):

        self.preds_key = preds_key
        self.target_key = target_key
        self.human_probs_for_human = []
        self.human_probs_for_spoof = []
        self.softmax = nn.Softmax(dim=1)
        self.metric_fn = metric_fn
        
    def on_batch_end(self, i, state) -> None:
        if state.core.mode == "val":
            preds = state.core.out[self.preds_key]
            preds = self.softmax(preds).cpu().numpy()
            targets = state.core.batch[self.target_key].cpu().numpy()
            tmp = preds[targets == 0, 0]
            if len(tmp) > 0:
                self.human_probs_for_human.append(tmp)
            tmp = preds[targets == 1, 0]
            if len(tmp) > 0:
                self.human_probs_for_spoof.append(tmp)
    
    def on_epoch_end(self, epoch, state) -> None:
        if state.core.mode == "val":
            print("_"*50)
            print("Metric computation")
            self.human_probs_for_human = np.hstack(self.human_probs_for_human)
            self.human_probs_for_spoof = np.hstack(self.human_probs_for_spoof)
            eer, threshold = self.metric_fn(self.human_probs_for_human, self.human_probs_for_spoof)
            self.human_probs_for_human = []
            self.human_probs_for_spoof = []
            print('score 1 (eer): {0:.7f} threshold (?): {1:.4f}'.format(
                eer, threshold))


In [13]:
keker = Keker(model=model,
              dataowner=dataowner,
              criterion=criterion,
              step_fn=step_fn,                    # previosly defined step function
              target_key="label",                 # remember, we defined it in the reader_fn for DataKek? 
              opt=torch.optim.Adam,               # optimizer class. if note specifiyng, 
                                                  # an SGD is using by default
              opt_params={"weight_decay": 1e-5},
              callbacks=[ScoreCallback('preds', 'label', compute_err)],
                 metrics={"acc": accuracy})

In [14]:
keker.kek(lr=1e-4,
          epochs=25,
          sched=torch.optim.lr_scheduler.MultiStepLR,       # pytorch lr scheduler class
          sched_params={"milestones":torch.tensor([12, 20]), "gamma": 0.1,},
         cp_saver_params={
              "savedir": "./checkpoints",
         "metric":"acc",
         "mode":'max'})  # schedulres kwargas in dict format

Epoch 1/25: 100% 2083/2084 [07:29<00:00,  7.50it/s, loss=0.2698]__________________________________________________
Metric computation
score 1 (eer): 18.0491042 threshold (?): 0.3863
Epoch 1/25: 100% 2084/2084 [07:54<00:00,  4.39it/s, loss=0.2698, val_loss=0.4317, acc=0.8282]
Epoch 2/25: 100% 2083/2084 [07:31<00:00,  7.43it/s, loss=0.1880]__________________________________________________
Metric computation
score 1 (eer): 21.6323822 threshold (?): 0.2845
Epoch 2/25: 100% 2084/2084 [07:57<00:00,  4.37it/s, loss=0.1880, val_loss=0.5258, acc=0.7789]
Epoch 3/25: 100% 2083/2084 [07:27<00:00,  7.32it/s, loss=0.1937]__________________________________________________
Metric computation
score 1 (eer): 21.4996682 threshold (?): 0.5062
Epoch 3/25: 100% 2084/2084 [07:52<00:00,  4.41it/s, loss=0.1937, val_loss=0.6064, acc=0.7846]
Epoch 4/25: 100% 2083/2084 [07:32<00:00,  7.36it/s, loss=0.1125]__________________________________________________
Metric computation
score 1 (eer): 16.5228932 threshold (?

In [15]:
torch.save(model.state_dict(), "checkpoints/weights.pt")

In [16]:
keker.kek(lr=1e-5,
          epochs=3,
         cp_saver_params={
              "savedir": "./checkpoints",
         "metric":"acc",
         "mode":'max'})  # schedulres kwargas in dict format

Epoch 1/3: 100% 2083/2084 [08:12<00:00,  7.37it/s, loss=0.0035]__________________________________________________
Metric computation
score 1 (eer): 21.1015262 threshold (?): 0.1394
Epoch 1/3: 100% 2084/2084 [08:39<00:00,  4.01it/s, loss=0.0035, val_loss=0.8843, acc=0.7858]
Epoch 2/3: 100% 2083/2084 [08:04<00:00,  7.11it/s, loss=0.0093]__________________________________________________
Metric computation
score 1 (eer): 20.4379562 threshold (?): 0.0706
Epoch 2/3: 100% 2084/2084 [08:31<00:00,  4.07it/s, loss=0.0093, val_loss=0.9589, acc=0.7904]
Epoch 3/3: 100% 2083/2084 [08:05<00:00,  7.34it/s, loss=0.0050]__________________________________________________
Metric computation
score 1 (eer): 21.0351692 threshold (?): 0.0199
Epoch 3/3: 100% 2084/2084 [08:33<00:00,  4.06it/s, loss=0.0050, val_loss=1.1948, acc=0.7756]

Checkpoint	acc
checkpoints/checkpoint.2.h5	0.790438
checkpoints/checkpoint.1.h5	0.785809
checkpoints/checkpoint.3.h5	0.775557


In [ ]:
keker = Keker(model=model,
              dataowner=dataowner,
              criterion=criterion,
              step_fn=step_fn,                    # previosly defined step function
              target_key="label",                 # remember, we defined it in the reader_fn for DataKek? 
              opt=torch.optim.Adam,               # optimizer class. if note specifiyng, 
                                                  # an SGD is using by default
              opt_params={"weight_decay": 1e-4},
              callbacks=[ScoreCallback('preds', 'label', compute_err)],
                 metrics={"acc": accuracy})
keker.kek(lr=1e-5,
          epochs=3,
         cp_saver_params={
              "savedir": "./checkpoints",
         "metric":"acc",
         "mode":'max'})  # schedulres kwargas in dict format

Epoch 1/3: 100% 2083/2084 [07:58<00:00,  6.63it/s, loss=0.0091]__________________________________________________
Metric computation
score 1 (eer): 19.5089582 threshold (?): 0.0397
Epoch 1/3: 100% 2084/2084 [08:26<00:00,  4.12it/s, loss=0.0091, val_loss=1.0375, acc=0.7924]
Epoch 2/3:  84% 1756/2084 [06:59<01:13,  4.46it/s, loss=0.0020]